实现复杂的网络的标准方式：  
**Block**：描述单/多层神经网络 组成的 组件/模型本身  
![Block 图片解释](https://github.com/Matthew-Hu-cmd/D2L_torch_tutorial/blob/main/data/imge4mkd/block_pic.png)  

block用class来表示，其任何一个子类都必须定义一个将其输入->输出的前向传播函数，并储存必要的参数。  
我们只需要考虑前向传播的必要参数，反向传播函数（在类自己实现）的必要参数已经在自动微分的操作中实现了  



In [6]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

# 实例化nn.Sequential，定义了一种表示块的类Module
# Sequential维护的是一个Module组成的有序列表
# 两个全连接层都是Linear类的实例，Linear类是Module的子类

tensor([[ 1.7941e-01,  1.1561e-01, -1.2792e-01, -1.2199e-01,  1.1207e-01,
         -1.0247e-01, -2.1515e-01, -7.7495e-02,  1.0695e-01,  4.9266e-02],
        [ 2.1666e-01, -3.6611e-02, -2.5776e-02, -6.5016e-02, -1.1341e-02,
         -5.2351e-02,  6.4909e-05, -6.6460e-02,  2.7591e-01,  1.1087e-01]],
       grad_fn=<AddmmBackward>)

自定义块的基本功能：  
1. 输入数据，作为前向传播的参数  
2. 前向传播函数，生成输出（输入输出可以形状不同）  
3. 计算输出关于输入的梯度（backward自动发生）  
4. 储存、访问forward的参数  
5. 根据需要初始化模型参数

In [9]:
class MLP(nn.Module):   #自定义一个MLP，是nn.Module的子类
    # 初始化声明函数，定义需要的类和参数
    def __init__(self): 
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        super().__init__()
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        self.hidden = nn.Linear(20, 256)  # 隐藏层，存在类的成员变量中
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X))) #隐藏层激活之后久直接放输出里面
# 用法
# 实例化这个类，每次正向传播的时候调用这些层
net = MLP()
net(X)

tensor([[ 0.0980,  0.0757,  0.0250,  0.2283, -0.1434, -0.1496,  0.1276, -0.0633,
          0.0514, -0.0061],
        [ 0.1546,  0.0632,  0.0003, -0.0225, -0.1284, -0.0496,  0.0938,  0.0042,
          0.0047, -0.0956]], grad_fn=<AddmmBackward>)

In [ ]:
# 标准化的自己在前向传播添加一些计算
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变，不给梯度，不参与训练
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
        # 只是实例一下如何集成任意的代码进去